In [11]:
from premise import *
from datapackage import Package
import brightway2 as bw

In [12]:
bw.projects.set_current("new")

In [13]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2020},
    {"model": "image", "pathway":"SSP2-Base", "year": 2030},
    {"model": "image", "pathway":"SSP2-Base", "year": 2040},
    {"model": "image", "pathway":"SSP2-Base", "year": 2050},
]

fp = "/Users/romain/GitHub/RTE_scenarios/datapackage.json"

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[
            Package(fp)
        ]
)
ndb.update_external_scenario()


premise v.(1, 4, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

Scenario no.:  15



//////////////////// EXTRACTING SOURCE DATABASE ////////////////////
Done!

////////////////// IMPORTING DEFAULT INVENTORIES ///////////////////
Done!

/////////////////////// EXTRACTING IAM DATA ////////////////////////
Done!

//////////////// IMPORTING USER-DEFINED INVENTORIES ////////////////
Extracted 1 worksheets in 0.00 seconds
Create custom markets.
0 FR Production|Gaseous fuel|CNG
[]
0 FR Production|Gaseous fuel|CNG
[]
0 FR Production|Gaseous fuel|LNG
[]
0 FR Production|Gaseous fuel|LNG
[]
0 FR Production|Gaseous fuel|Biomethane
[]
0 FR Production|Gaseous fuel|Biomethane
[]
0 FR Production|Gaseous fuel|Low to high
[]
0 FR Production|Gaseous fuel|Low to high
[]
0 FR Production|Liquid fuel|Diesel|Diesel
[]
0 FR Production|Liquid fuel|Diesel|Diesel
[]
0 FR Production|Liquid fuel|Diesel|Biodiesel
[]
0 FR Production|Liquid fuel|Diesel|Biodiesel
[]
0 FR Production|Liquid fuel|Petrol|Petrol
[]
0 FR Production|Liquid fuel|Petrol|Petrol
[]
0 FR Production|Liquid fuel|Petrol|Bioethanol


In [14]:
ndb.write_db_to_brightway(["RTE_2020", "RTE_2030", "RTE_2040", "RTE_2050"])

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!


KeyError: "Can't find linking activity for exchange:\n{'amount': 0.000233439861595437,\n 'location': 'World',\n 'name': 'market for electricity, medium voltage, FE2050',\n 'product': 'electricity, medium voltage',\n 'type': 'technosphere',\n 'unit': 'kilowatt hour'}"

In [8]:
if a:
    print("yeah")